In [1]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import os

In [2]:
def fetch_series_data(series_id, start_year, end_year, api_key):

    url = (f"https://api.stlouisfed.org/fred/series/observations"
           f"?series_id={series_id}"
           f"&observation_start={start_year}"
           f"&observation_end={end_year}"
           f"&api_key={api_key}"
           f"&limit=13&file_type=json&sort_order=desc")

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code}")
        print(response.text)
        return

    data = response.json()
    
    # Process and print the results with a friendly header based on Series ID
    print()
    series_labels = {
        "FEDFUNDS": "Federal Funds Rate: ",
        "ICSA": "Initial Claims for Unemployment Insurance: ",
        "RSAFS": "Retail Sales: ",
        "UNRATE": "Unemployment Rate: ",
        "GDP": "Gross Domestic Product: ",
        "DGORDER": "Durable Goods Orders: ",
        "INDPRO": "Industrial Production: ",
        "PCE": "Personal Consumption Expenditures: "
    }
    
    label = series_labels.get(series_id, "Series ID: ")
    print(label)

    observations = data.get('observations', [])
    if observations:
        latest = observations[0]  # The first element is the latest due to descending sort
        print(f"{latest['value']} on {latest['date']}")

        # Ensure there are at least 2 observations to calculate month-over-month change
        if len(observations) > 1:
            previous = observations[1]
            latest_value = float(latest['value'])
            previous_value = float(previous['value'])
            change = latest_value - previous_value
            percentage_change = (change / previous_value) * 100
            print(f"Change from previous month: {change:.2f} ({percentage_change:.2f}%)")
        else:
            print("Not enough data to calculate month-over-month change")

        # Calculate 12-month change if there's enough data
        if len(observations) >= 13:
            year_ago = observations[12]  # 12th element is data from 12 months ago
            latest_value = float(latest['value'])
            year_ago_value = float(year_ago['value'])
            year_change = latest_value - year_ago_value
            year_percentage_change = (year_change / year_ago_value) * 100
            print(f"Change from 12 months ago: {year_change:.2f} ({year_percentage_change:.2f}%)")
        else:
            print("Not enough data to calculate 12-month change")
    else:
        print("No data available in the specified date range")

    print(f"FRED Series ID: {series_id}")
    print()


In [3]:

def get_fred():

    # Read the API key from the environment variable
    api_key = os.getenv('API_KEY')
    if api_key:
        print("API Key found.")
    else:
        print("API Key not set.")
    
    series_ids = ["FEDFUNDS", "ICSA", "RSAFS", "UNRATE", "GDP", "DGORDER", "INDPRO", "PCE"]
    now = datetime.now()
    current_month = now.strftime("%Y-%m-%d")
    one_year_ago = (now - timedelta(days=365)).strftime("%Y-%m-%d")

    for series_id in series_ids:
        fetch_series_data(series_id, one_year_ago, current_month, api_key)

